```bash
## environment
pip3 install kafka-python

zookeeper-server-start.sh $KAFKA_HOME/config/zookeeper.properties
kafka-server-start.sh $KAFKA_HOME/config/server.properties
kafka-topics.sh --create --zookeeper localhost:2181 --replication-factor 1 --partitions 1 --topic bitcoin
kafka-topics.sh --create --zookeeper localhost:2181 --replication-factor 1 --partitions 1 --topic news
```

In [1]:
from kafka import KafkaProducer
from json import dumps
import time
import json

from urllib.request import urlopen
import requests
import datetime

In [2]:
# to send sentiment before to producer
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
vader = SentimentIntensityAnalyzer()

In [3]:
from pyspark import SparkContext
from pyspark import SparkConf

ip = 'localhost'
producer = KafkaProducer(bootstrap_servers=[ip+':9092'],
                         api_version=(0,11,5),
                         value_serializer=lambda x: 
                         dumps(x).encode('utf-8'))

news_api_key = "b3aae8a2a9f34d82ae214aea72d0f2b6"

In [6]:
# Simulate when get new price and news
day = 0
starttime = (datetime.datetime.now() - datetime.timedelta(days=2*day+1)).strftime("%Y-%m-%d")
endtime =(datetime.datetime.now() - datetime.timedelta(days=2*day)).strftime("%Y-%m-%d")

print("From: ",starttime)
print("To: ", endtime)

bitcoin = requests.get("https://api.coindesk.com/v1/bpi/historical/close.json?start="+starttime+"&end="+endtime)
news = requests.get("https://newsapi.org/v2/everything?q=bitcoin&from="+starttime+"&to="+endtime+"&sortBy=publishedAt&language=en&apiKey=b3aae8a2a9f34d82ae214aea72d0f2b6")

bitcoin_kv = bitcoin.json()['bpi']
for k,v in bitcoin_kv.items():
    kv_message = {"date":k, "price":v, "type":"bitcoin"}
    producer.send("bitcoin",kv_message)
    print(kv_message)

for i in news.json()["articles"]:
    if i['title'] != None:
        # ------- sentiment score -----------
        sentiment_score = vader.polarity_scores(i['title'])["compound"]
        sentiment_score_digits = f'{sentiment_score:.4f}'
        kv = {"date":starttime, "sentiment":str(sentiment_score_digits), "title":i['title'],"type":"news"}
        producer.send("news",kv)
        print(kv)
    break

From:  2021-10-21
To:  2021-10-22
{'date': '2021-10-21', 'price': 62201.9633, 'type': 'bitcoin'}
{'date': '2021-10-21', 'sentiment': '-0.9001', 'title': "GOP congressman: Capitol riot defendants are being treated worse than 'blood thirsty murderers' at Guantanamo", 'type': 'news'}


## When gathering training set:

```python
# To gather training set
# Send news/bitcoin price until today to kafka

historical_days = 7


for day in historical_days(3):
    starttime = (datetime.datetime.now() - datetime.timedelta(days=2*day+1)).strftime("%Y-%m-%d")
    endtime =(datetime.datetime.now() - datetime.timedelta(days=2*day)).strftime("%Y-%m-%d")
    
    print(starttime)
    print(endtime)
    
    bitcoin = requests.get("https://api.coindesk.com/v1/bpi/historical/close.json?start="+starttime+"&end="+endtime)
    bitcoin_kv = bitcoin.json()['bpi']
    for k,v in bitcoin_kv.items():
        kv_message = {"date":k, "price":v, "type":"bitcoin"}
        producer.send("bitcoin",kv_message)
        print(kv_message)
        
    news = requests.get("https://newsapi.org/v2/everything?q=bitcoin&from="+starttime+"&to="+endtime+"&sortBy=publishedAt&language=en&apiKey=b3aae8a2a9f34d82ae214aea72d0f2b6")
    for i in news.json()["articles"]:
        if i['title'] != None:

            # ------- sentiment score -----------
            sentiment_score = vader.polarity_scores(i['title'])["compound"]
            sentiment_score_digits = f'{sentiment_score:.4f}'
            kv = {"date":starttime, "sentiment":str(sentiment_score_digits), "title":i['title'],"type":"news"}
            producer.send("news",kv)
            print(kv)

    print("-"*30)
```